In [ ]:
%matplotlib inline
#這是juoyter notebook的magic word˙

import matplotlib
import matplotlib.pyplot as plt
from IPython import display

In [ ]:
import os
#判斷是否在jupyter notebook上
def is_in_ipython():
    "Is the code running in the ipython environment (jupyter including)"
    program_name = os.path.basename(os.getenv('_', ''))

    if ('jupyter-notebook' in program_name or # jupyter-notebook
        'ipython'          in program_name or # ipython
        'jupyter' in program_name or  # jupyter
        'JPY_PARENT_PID'   in os.environ):    # ipython-notebook
        return True
    else:
        return False


#判斷是否在colab上
def is_in_colab():
    if not is_in_ipython(): return False
    try:
        from google import colab
        return True
    except: return False

#判斷是否在kaggke_kernal上
def is_in_kaggle_kernal():
    if 'kaggle' in os.environ['PYTHONPATH']:
        return True
    else:
        return False

if is_in_colab():
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
os.environ['TRIDENT_BACKEND'] = 'pytorch'

if is_in_kaggle_kernal():
    os.environ['TRIDENT_HOME'] = './trident'
    
elif is_in_colab():
    os.environ['TRIDENT_HOME'] = '/content/gdrive/My Drive/trident'

#為確保安裝最新版 
!pip uninstall tridentx -y
!pip install tridentx --upgrade
!pip install pydicom --upgrade
!pip install python-gdcm  --upgrade

import json
import copy
import numpy as np
#調用trident api
import trident as T
from trident import *
from trident.models import resnet,efficientnet
import random
from tqdm import tqdm
import pydicom
import gdcm
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

In [ ]:
dicom_images=glob.glob('../input/siim-covid19-detection/train/*/*/*.dcm')

print(len(dicom_images))

In [ ]:

resize_fn=Resize((608,608))
make_dir_if_need('./train_images')
images_dict=OrderedDict()
for i in tqdm(range(len(dicom_images))):
    img=dicom_images[i]
    img_path=img.replace('../input/siim-covid19-detection/train/','')
    #將路徑分離成資料夾、檔案名稱以及附檔名
    folder,filename,ext=split_path(img_path)
    #將資料夾字串分割
    img_folders=folder.split('/')
    #基於第一層資料夾以及圖檔名作為key
    images_dict[img_folders[0]+'-'+filename]=OrderedDict()
    
    #取出dicom圖檔
    pixels=dicom2array(img)
    #縮放為608x608
    resize_pixels=resize_fn(pixels)
    #將縮放後檔案
    array2image(resize_pixels).save('./train_images/{0}.png'.format(img_folders[0]+'-'+filename))
    images_dict[img_folders[0]+'-'+filename]['img_path']='./train_images/{0}.png'.format(img_folders[0]+'-'+filename)
    images_dict[img_folders[0]+'-'+filename]['shape']=pixels.shape
    del pixels
    del resize_pixels
    
print(len(images_dict))
print(list(images_dict.items())[:5])

In [ ]:
import json
f=open('./train_images/image.json','w',encoding='utf-8-sig')
f.write(json.dumps(images_dict))
    

In [ ]:
import json
f=open('./train_images/image.json','r',encoding='utf-8-sig')
images_dict=json.loads(f.read())
print(len(images_dict))

In [ ]:
#這裡的id是圖片的資料夾第一層
import pandas as pd
df_train_study=pd.read_csv('../input/siim-covid19-detection/train_study_level.csv', index_col='id')
df_train_study = df_train_study.idxmax(axis=1).to_frame(name='case')
df_train_study.index = df_train_study.index.str.replace('_study', '')
df_train_study

In [ ]:
df_train_image=pd.read_csv('../input/siim-covid19-detection/train_image_level.csv', index_col='id')
#df_train_image = df_train_image.merge(df_train_study, left_on='StudyInstanceUID', right_index=True)
df_train_image

In [ ]:
print([img for img in dicom_images if '000a312787f2' in img])

In [ ]:
df_train_image = df_train_image.merge(df_train_study, left_on='StudyInstanceUID', right_index=True)
df_train_image

In [ ]:
label_dict={'Negative for Pneumonia':0, 'Typical Appearance':1,'Indeterminate Appearance':2,'Atypical Appearance':3}

for i, row in df_train_image.iterrows():
    image_id = row['StudyInstanceUID']+'-'+ row.name.replace('_image','')
    #if image_id in images_dict:
    #    print(image_id,'not in images_dict')


    value = row['boxes']
    scale=608/builtins.max(images_dict[image_id]['shape'][:2])
    if isinstance(value, float): # nan
        images_dict[image_id]['orig_boxes']=[]
        images_dict[image_id]['boxes']=[]
        images_dict[image_id]['scale']=scale
        continue
 
    else:
        values = eval(value)
        current_boxes=[]
        adj_boxes=[]
        
        for val in values:
            current_boxes.append([float(val['x']), float(val['y']),float(val['x'] + val['width']),float(val['y'] + val['height']),float(label_dict[row['case']])])
            adj_boxes.append([float(val['x'])*scale, float(val['y'])*scale,float(val['x'] + val['width'])*scale,float(val['y'] + val['height'])*scale,float(label_dict[row['case']])])
        images_dict[image_id]['orig_boxes']=current_boxes
        images_dict[image_id]['boxes']=adj_boxes
        images_dict[image_id]['scale']=scale


f=open('./train_images/image.json','w',encoding='utf-8-sig')
f.write(json.dumps(images_dict))
print(list(images_dict.items())[:5])
print(len(images_dict))

In [ ]:
!zip -r output.zip ./train_images/


In [ ]:
#取回你的模型的最簡單方法
from IPython.display import FileLink
FileLink('./output.zip')

In [ ]:
!rm -rf  ./train_images/*